In [1]:
# in a terminal run
# > make run-redis NS=train
# > make run-redis NS=test
# to allow access to the train and test namespaces

In [2]:
import os
import sys
import pandas as pd
import numpy as np

In [3]:
sys.path.append("..")
os.environ["USER_PATH"] = "/mnt/d/workspace/clotho/userdata/"
MODEL_OUTPUT_BASE = "/mnt/d/workspace/clotho/notebooks"
MODEL_OUTPUT_CP = os.path.join(MODEL_OUTPUT_BASE, "checkpoints")

In [4]:
from misc.redis import set_redis_slow_mode
from misc.util import highest_number
from model.datagenerator import create_train_test
from system.namespace.store import get_namespace

In [5]:
import torch

is_cuda = torch.cuda.is_available()
is_cuda

True

In [6]:
set_redis_slow_mode("never")
ns_test = get_namespace("test")
ns_train = get_namespace("train")
now = pd.Timestamp("2022-12-17", tz="UTC")
train_plan = [
    {
        "left": {"mode": "valid", "flip_pc": 0.5},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": 20,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": 10,
        "last_epoch": None,
        "weight": 100,
    },
    {
        "left": {"mode": "valid", "flip_pc": 0.5},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": 10,
        "last_epoch": None,
        "weight": 100,
    },
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "path", "flip_pc": 0.0},
        "min_text_length": 20,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": None,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "path", "flip_pc": 0.0},
        "min_text_length": 20,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": None,
        "weight": 40,
    },
     {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "path", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": 5,
        "last_epoch": None,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "path", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": 5,
        "last_epoch": None,
        "weight": 40,
    },
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": None,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": None,
        "weight": 40,
    },
    {
        "left": {"mode": "valid", "flip_pc": 0.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0,
        "first_epoch": 15,
        "last_epoch": None,
        "weight": 50,
    }
]
eval_plan = [
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": 20,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": 20,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "weight": 40,
    },
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "weight": 40,
    },
]
ttgen = create_train_test(
    train_ns=ns_train,
    train_validation_ns=ns_train,
    test_ns=ns_test,
    test_validation_ns=ns_test,
    train_learning_plan=train_plan,
    train_val_learning_plan=eval_plan,
    test_learning_plan=eval_plan,
    test_val_learning_plan=eval_plan,
    batch_size=4 if is_cuda else 8,
    epoch_batches=5000 if is_cuda else 500,
    train_val_size=10000 if is_cuda else 1000,
    test_size=10000 if is_cuda else 1000,
    test_val_size=10000 if is_cuda else 1000,
    compute_batch_size=100 if is_cuda else 100,
    now=now)

In [7]:
import torch.nn as nn
from transformers import DistilBertTokenizer, DistilBertModel

In [8]:
device = torch.device("cuda") if is_cuda else torch.device("cpu")
device

device(type='cuda')

In [9]:
from typing import Literal, TypedDict

ProviderRole = Literal["child", "parent"]

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
EMBED_SIZE = 768

TokenizedInput = TypedDict('TokenizedInput', {
    "input_ids": torch.Tensor,
    "attention_mask": torch.Tensor,
})


def tokens(texts: list[str]) -> TokenizedInput:
    res = tokenizer(texts.tolist(), return_tensors="pt", padding=True, truncation=True)
    return {k: v.to(device) for k, v in res.items()}


class Noise(nn.Module):
    def __init__(self, std: float = 1.0, p: float = 0.5) -> None:
        super().__init__()
        self._std = std
        self._p = p
        self._dhold = nn.Parameter(torch.Tensor([0.0]), requires_grad=False)

    def set_std(self, std: float) -> None:
        self._std = std

    def get_std(self) -> float:
        return self._std

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if not self.training:
            return x
        prob = torch.rand(size=x.shape, device=self._dhold.device) < self._p
        gauss = torch.normal(
            mean=0.0, std=self._std, size=x.shape, device=self._dhold.device)
        return x + prob * gauss


class Model(nn.Module):
    def __init__(self, version: int) -> None:
        super().__init__()
        self._bert_parent = DistilBertModel.from_pretrained(
            "distilbert-base-uncased")
        self._bert_child = DistilBertModel.from_pretrained(
            "distilbert-base-uncased")
        if version in (1, 3, 4, 6):
            self._pdense: nn.Sequential | None = nn.Sequential(
                nn.Linear(EMBED_SIZE, EMBED_SIZE),
                nn.Dropout(p=0.2),
                nn.ReLU(),
                nn.Linear(EMBED_SIZE, EMBED_SIZE))
            self._cdense: nn.Sequential | None = nn.Sequential(
                nn.Linear(EMBED_SIZE, EMBED_SIZE),
                nn.Dropout(p=0.2),
                nn.ReLU(),
                nn.Linear(EMBED_SIZE, EMBED_SIZE))
        else:
            self._pdense = None
            self._cdense = None
        if version < 4 or noise > 5:
            self._noise = None
        else:
            self._noise = Noise(std=1.0, p=0.2)
        if version < 2 or version > 4:
            self._cos = None
        else:
            self._cos = torch.nn.CosineSimilarity()
        if version < 6:
            self._agg = "cls"
        else:
            self._agg = "mean"
        self._version = version

    def set_epoch(self, epoch: int) -> None:
        noise = self._noise
        if noise is not None:
            noise.set_std(1 / (1.2 ** epoch))

    def get_version(self) -> int:
        return self._version
    
    def get_agg(self, lhs: torch.Tensor) -> torch.Tensor:
        if self._agg == "cls":
            return lhs[:, 0]
        if self._agg == "mean":
            return torch.mean(lhs, dim=1)
        raise ValueError(f"unknown aggregation: {self._agg}")

    def get_parent_embed(
            self,
            input_ids: torch.Tensor,
            attention_mask: torch.Tensor) -> torch.Tensor:
        outputs_parent = self._bert_parent(
            input_ids=input_ids, attention_mask=attention_mask)
        out = self.get_agg(outputs_parent.last_hidden_state)
        if self._pdense is not None:
            out = self._pdense(out)
        if self._noise is not None:
            out = self._noise(out)
        return out

    def get_child_embed(
            self,
            input_ids: torch.Tensor,
            attention_mask: torch.Tensor) -> torch.Tensor:
        outputs_child = self._bert_child(
            input_ids=input_ids, attention_mask=attention_mask)
        out = self.get_agg(outputs_child.last_hidden_state)
        if self._cdense is not None:
            out = self._cdense(out)
        if self._noise is not None:
            out = self._noise(out)
        return out

    def forward(self, x: dict[ProviderRole, TokenizedInput]) -> torch.Tensor:
        parent_cls = self.get_parent_embed(
            input_ids=x["parent"]["input_ids"],
            attention_mask=x["parent"]["attention_mask"])
        child_cls = self.get_child_embed(
            input_ids=x["child"]["input_ids"],
            attention_mask=x["child"]["attention_mask"])
        if self._cos is not None:
            return self._cos(parent_cls, child_cls).reshape([-1, 1])
        batch_size = parent_cls.shape[0]
        return torch.bmm(
            parent_cls.reshape([batch_size, 1, -1]),
            child_cls.reshape([batch_size, -1, 1])).reshape([-1, 1])


class TrainingHarness(nn.Module):
    def __init__(self, model: Model) -> None:
        super().__init__()
        self._model = model
        self._softmax = nn.Softmax(dim=1)
        self._loss = nn.BCELoss()

    def get_version(self) -> int:
        return self._model.get_version()

    def forward(
            self,
            left: TokenizedInput,
            right: TokenizedInput,
            labels: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        out_left = self._model(left)
        out_right = self._model(right)
        preds = self._softmax(torch.hstack((out_left, out_right)))
        return preds, self._loss(preds, labels)

In [ ]:
def get_filename(
        harness: TrainingHarness,
        *,
        is_final: bool,
        ftype: str,
        epoch: int | None,
        ext: str = ".pkl") -> tuple[str, str, str]:
    folder = MODEL_OUTPUT_BASE if is_final else MODEL_OUTPUT_CP
    postfix = "_lg" if is_cuda else ""
    version_tag = f"_v{harness.get_version()}"
    out_pre = f"{ftype}{version_tag}{postfix}_"
    out_post = ext
    return (
        os.path.join(
            folder,
            f"{out_pre}{"" if epoch is None else epoch}{out_post}"),
        out_pre,
        out_post,
        folder,
    )

In [10]:
from torch.optim import AdamW

model = Model(version=6)
model.to(device)
harness = TrainingHarness(model)
harness.to(device)

FORCE_RESTART = False

_, spre, spost, sfolder = get_filename(harness, is_final=False, ftype="harness", epoch=None)
mprev = highest_number(os.listdir(sfolder), prefix=spre, postfix=spost)
if not FORCE_RESTART and mprev is not None:
    prev_fname, prev_epoch = mprev
    harness.load_state_dict(torch.load(os.path.join(folder, prev_fname), map_location=device))
    epoch_offset = prev_epoch + 1
else:
    epoch_offset = 0

optimizer = AdamW(harness.parameters(), lr=5e-5)
mprev, epoch_offset

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- T

(None, 0)

In [11]:
from transformers import get_scheduler
# from tqdm.notebook import tqdm
from tqdm.auto import tqdm
import evaluate
import time

def compute(df):
    plefts = tokens(df["parent_left"])
    clefts = tokens(df["child_left"])
    prights = tokens(df["parent_right"])
    crights = tokens(df["child_right"])
    labels = torch.tensor([~df["correct_is_right"], df["correct_is_right"]], dtype=torch.float32).T.to(device)
    return harness({"parent": plefts, "child": clefts}, {"parent": prights, "child": crights}, labels)

num_epochs = max((120 if is_cuda else 10) - epoch_offset, 3)
num_training_steps = num_epochs * ttgen.get_epoch_train_size()
warmup = 10000 if is_cuda else 10
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=warmup,
    num_training_steps=num_training_steps - warmup)
ttgen.set_epoch(epoch_offset)

for _ in range(num_epochs):
    epoch = ttgen.get_epoch()
    print(f"epoch {epoch}")
    real_time = time.monotonic()
    
    model.train()
    harness.train()
    model.set_epoch(epoch)
    metric_train = evaluate.load("accuracy")
    train_loss = []
    first = True
    with tqdm(desc="train", total=ttgen.get_epoch_train_size()) as progress_bar:
        for train_df in ttgen.train_dfs():
            preds, loss = compute(train_df)
            train_loss.append(loss.item())
            loss.backward()

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(train_df.shape[0])
            
            predictions = torch.argmax(preds, dim=-1)
            metric_train.add_batch(predictions=predictions, references=train_df["correct_is_right"].astype(int))
            if first:
                # display(train_df)
                first = False

    torch.save(harness.state_dict(), get_filename(harness, is_final=False, ftype="harness", epoch=epoch))
            
    model.eval()
    harness.eval()
    with torch.no_grad():
        metric_val_train = evaluate.load("accuracy")
        train_val_loss = []
        with tqdm(desc="train val", total=ttgen.get_epoch_train_validation_size()) as progress_bar:
            for train_validation_df in ttgen.train_validation_dfs():
                preds, loss = compute(train_validation_df)
                train_val_loss.append(loss.item())
                predictions = torch.argmax(preds, dim=-1)
                metric_val_train.add_batch(
                    predictions=predictions, references=train_validation_df["correct_is_right"].astype(int))
                progress_bar.update(train_validation_df.shape[0])
        
        metric_test = evaluate.load("accuracy")
        test_loss = []
        with tqdm(desc="test", total=ttgen.get_epoch_test_size()) as progress_bar:
            for test_df in ttgen.test_dfs():
                preds, loss = compute(test_df)
                test_loss.append(loss.item())
                predictions = torch.argmax(preds, dim=-1)
                metric_test.add_batch(
                    predictions=predictions, references=test_df["correct_is_right"].astype(int))
                progress_bar.update(test_df.shape[0])
        
        print(f"train: {metric_train.compute()} loss: {np.mean(train_loss)}")
        print(f"train val: {metric_val_train.compute()} loss: {np.mean(train_val_loss)}")
        print(f"test: {metric_test.compute()} loss: {np.mean(test_loss)}")
    ttgen.advance_epoch()
    print(f"epoch time: {(time.monotonic() - real_time) / 60.0:.2f}min")

epoch 0


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

train: {'accuracy': 0.51025} loss: 22.655790788561532
train val: {'accuracy': 0.5311} loss: 0.6934446441173554
test: {'accuracy': 0.4985} loss: 0.7083818842530251
epoch time: 84.80min
epoch 1


...

train: {'accuracy': 0.7097} loss: 0.5258638797266408
train val: {'accuracy': 0.6913} loss: 0.5614567510306835
test: {'accuracy': 0.6159} loss: 0.6636211219191551
epoch time: 19.28min
epoch 99


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

train: {'accuracy': 0.70885} loss: 0.5282410882648081
train val: {'accuracy': 0.6928} loss: 0.560701883161068
test: {'accuracy': 0.6391} loss: 0.6381455819368362
epoch time: 19.34min


In [12]:
torch.save(model.state_dict(), get_filename(harness, is_final=True, ftype="model", epoch=None)[0])
torch.save(harness.state_dict(), get_filename(harness, is_final=True, ftype="harness", epoch=None)[0])
torch.save(optimizer.state_dict(), get_filename(harness, is_final=True, ftype="optimizer", epoch=None)[0])

In [13]:
ttgen.reset()
model.eval()
harness.eval()
dfs = []
with torch.no_grad():
    metric_val_test = evaluate.load("accuracy")
    test_val_loss = []
    with tqdm(desc="test val", total=ttgen.get_epoch_test_validation_size()) as progress_bar:
        for test_val_df in ttgen.test_validation_dfs():
            preds, loss = compute(test_val_df)
            test_val_loss.append(loss.item())
            predictions = torch.argmax(preds, dim=-1)
            metric_val_test.add_batch(
                predictions=predictions, references=test_val_df["correct_is_right"].astype(int))
            cur_df = test_val_df.copy()
            cur_df["logit_left"] = preds[:, 0].cpu()
            cur_df["logit_right"] = preds[:, 1].cpu()
            cur_df["preds"] = predictions.cpu()
            cur_df["truth"] = test_val_df["correct_is_right"].astype(int)
            dfs.append(cur_df)
            progress_bar.update(test_val_df.shape[0])
print(f"test val: {metric_val_test.compute()} loss: {np.mean(test_val_loss)}")
validation_df = pd.concat(dfs)

test val:   0%|          | 0/10000 [00:00<?, ?it/s]

test val: {'accuracy': 0.6371} loss: 0.6438971089482307


In [14]:
validation_df.to_csv(
    get_filename(harness, is_final=True, ftype="validation", epoch=None, ext=".csv")[0])

In [15]:
validation_df[validation_df["preds"] == validation_df["truth"]].head()

,gen_name,parent_left,child_left,parent_right,child_right,sway_left,sway_right,correct_is_right,logit_left,logit_right,preds,truth
2,random--valid;(sw),Dungeons and Dragons,r/Kamikazebywords,"Question for the men, what's the worst part of...",When sitting wrong on your bike seat and cutti...,0.119203,8.807971e-01,True,0.386937,0.613063,1,1
3,random--valid;(sw),I always brush my teeth but need to floss more...,As an american all I can say is: why?,[removed],User profile checks out. Bet you're on a list ...,0.119203,8.807971e-01,True,0.115898,0.884102,1,1
0,*valid--random;(sw),And they taste delicious when chopped into hal...,"In Spain/Portugal we also cook them ""a feira"" ...",Seinfeld seems to be the one that paved the wa...,Wiping down gym equipment.,0.993307,6.692851e-03,False,0.539984,0.460016,0,0
1,*valid--!copy;(mtl:20),Yeah that's a thing. I spend my summer vacatio...,you ever done on going maintenance on a boat? ...,you ever done on going maintenance on a boat? ...,Yeah that's a thing. I spend my summer vacatio...,1.000000,1.562882e-18,False,0.547646,0.452354,0,0
2,*valid--!copy;(sw),She actually has a song where this is the poin...,"Yep, this one came to mind. She is incredible ...","Yep, this one came to mind. She is incredible ...",She actually has a song where this is the poin...,0.997527,2.472623e-03,False,0.627038,0.372962,0,0


In [16]:
validation_df[validation_df["preds"] != validation_df["truth"]].head()

,gen_name,parent_left,child_left,parent_right,child_right,sway_left,sway_right,correct_is_right,logit_left,logit_right,preds,truth
0,!copy--valid;(mtl:20),Gonna ride the baloney pony all the way to Was...,I'd literally take the law into my own hands.,I'd literally take the law into my own hands.,Gonna ride the baloney pony all the way to Was...,4.662937e-15,1.000000,True,0.527256,0.472744,0,1
1,*valid--!copy;(mtl:20),Men are still plenty selective about who they ...,"Men are selective, as in, they'll select from ...","Men are selective, as in, they'll select from ...",Men are still plenty selective about who they ...,9.998766e-01,0.000123,False,0.480113,0.519887,1,0
3,*valid--random;(mtl:20),The cigarette industry social lied about cigar...,covid vaccines can protect from covid,“Women get sex so easy!” “Man I bet no one wan...,You guys watch porn with sound on??? What a pr...,8.807971e-01,0.119203,False,0.184878,0.815122,1,0
1,*valid--!copy;(mtl:20),"Yeah that's a fair point, which is why I belie...",>The idea was for God's people to be such good...,>The idea was for God's people to be such good...,"Yeah that's a fair point, which is why I belie...",7.310586e-01,0.268941,False,0.462959,0.537041,1,0
1,!copy--valid;(mtl:20),I remember learning in 2nd grade that pizza wa...,The food pyramid -brought to you by big grain,The food pyramid -brought to you by big grain,I remember learning in 2nd grade that pizza wa...,0.000000e+00,1.000000,True,0.654011,0.345989,0,1


In [17]:
ttgen.reset()
model.eval()
harness.eval()
with torch.no_grad():
    count = 0
    for test_val_df in ttgen.test_validation_dfs():
        plefts = tokens(test_val_df["parent_left"])
        clefts = tokens(test_val_df["child_left"])
        prights = tokens(test_val_df["parent_right"])
        crights = tokens(test_val_df["child_right"])
        display(model.get_child_embed(
            clefts["input_ids"],
            clefts["attention_mask"]).cpu().numpy())
        display(model.get_child_embed(
            crights["input_ids"],
            crights["attention_mask"]).cpu().numpy())
        count += 1
        if count >= 5:
            break

tensor([[-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537]],
       device='cuda:0')

tensor([[-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537]],
       device='cuda:0')

tensor([[-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537]],
       device='cuda:0')

tensor([[-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537]],
       device='cuda:0')

tensor([[-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537]],
       device='cuda:0')

tensor([[-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537]],
       device='cuda:0')

tensor([[-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537]],
       device='cuda:0')

tensor([[-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537]],
       device='cuda:0')

tensor([[-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537]],
       device='cuda:0')

tensor([[-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537],
        [-0.3967,  0.3497,  0.7303,  ...,  0.3481,  0.4669,  0.4537]],
       device='cuda:0')